In [1]:
import pandas as pd 
import scipy.sparse as sparse
import implicit
import numpy as np

eslite_db = pd.read_csv('/Users/lucaschang/Desktop/誠品生活/eslite_ec_data.csv')

print(eslite_db.info())

FileNotFoundError: [Errno 2] No such file or directory: '/Users/lucaschang/Desktop/誠品生活/eslite_ec_data.csv'

In [ ]:
eslite_db['member_id'].value_counts()

eslite_db[['member_id']].sort_values(by=['member_id'])

In [3]:
eslite_db.sub_order_item_id.unique().shape

(262316,)

In [18]:
csr_matrix = sparse.csr_matrix(eslite_db[['member_id','product_id']].values)

In [20]:
csr_matrix.toarray().shape

(262316, 2)

In [10]:
df1 = (eslite_db.head(100).assign(new = 1)
         .drop_duplicates(subset=['product_id','member_id'])
         .pivot('product_id','member_id','new')
         .fillna(0)
         .astype(int))

In [35]:
print(type(df1))

print(df1.head(10))

print(df1.info)

print(df1.index)

<class 'pandas.core.frame.DataFrame'>
member_id   186795   334533   382875   434418   490200   617119   727488   \
product_id                                                                  
1673340           0        0        0        0        0        0        0   
2004135           0        0        0        0        0        0        0   
2306677           0        0        0        0        0        0        0   
2441919           0        0        0        0        0        0        0   
2473874           0        0        0        0        0        0        0   
2481219           0        0        0        0        0        0        0   
2515318           0        0        0        0        0        0        0   
2597098           0        0        0        0        0        0        0   
2604024           0        0        0        0        0        0        0   
2607960           0        0        0        0        0        0        0   

member_id   788704   802106   926088 

In [38]:
#person product table

In [39]:
mem_prod_df1 = (eslite_db.head(100).assign(new = 1)
         .drop_duplicates(subset=['member_id','product_id'])
         .pivot('member_id','product_id','new')
         .fillna(0)
         .astype(int))

In [40]:
print(type(mem_prod_df1))

print(mem_prod_df1.head(10))

print(mem_prod_df1.info)

print(mem_prod_df1.index)

<class 'pandas.core.frame.DataFrame'>
product_id  1673340  2004135  2306677  2441919  2473874  2481219  2515318  \
member_id                                                                   
186795            0        0        0        0        0        0        0   
334533            0        0        0        0        0        0        0   
382875            0        0        0        0        0        0        0   
434418            0        0        0        0        0        0        0   
490200            0        0        0        0        0        0        0   
617119            0        0        0        0        0        0        0   
727488            0        0        0        0        0        0        0   
788704            0        0        0        0        0        0        0   
802106            0        0        0        0        0        0        0   
926088            0        0        0        0        0        0        0   

product_id  2597098  2604024  2607960

In [12]:
df1_csr = sparse.csr_matrix(df1)

In [41]:
mem_prod_df1_csr = sparse.csr_matrix(mem_prod_df1)

In [15]:
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=50)

In [16]:
alpha = 15
data = (df1_csr * alpha).astype('double')

# Fit the model
model.fit(data)

In [36]:
product_id = 0
n_similar = 10

#？？model.user_factors要怎麼查的出來
person_vecs = model.user_factors
content_vecs = model.item_factors

#?
content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))

#?好像是做標準化？
scores = content_vecs.dot(content_vecs[product_id]) / content_norms

#?
top_idx = np.argpartition(scores, -n_similar)[-n_similar:]

#?zip 是幹嘛的，similar是相似度
similar = sorted(zip(top_idx, scores[top_idx] / content_norms[product_id]), key=lambda x: -x[1])

In [37]:
sorted(similar)

[(0, 1.0000001),
 (1, 1.0000001),
 (3, 0.24092342),
 (7, 0.24433595),
 (14, 0.30832234),
 (21, 0.22465049),
 (40, 0.19825564),
 (50, 0.217635),
 (63, 1.0),
 (65, 0.23746976)]

In [25]:
for content in similar:
    idx, score = content
    print(eslite_db.product_name.loc[eslite_db[['member_id','product_id']].product_id == idx].iloc[0])

IndexError: single positional indexer is out-of-bounds

In [ ]:
def recommend(person_id, mem_prod_df1_csr, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = mem_prod_df1_csr[person_id,:].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0
    # Get dot product of person vector and all content vectors（specific person all content vecs)
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray()
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    #?
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero ??
    recommend_vector = person_interactions * rec_vector_scaled   ##why miltiple rec_vector_scaled ??
    # Sort the indices of the content into order of best recommendations   ##argsort??
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store titles and scores
    titles = []
    scores = []

    for idx in content_idx:
        # Append titles and scores to the list
        titles.append(grouped_df.title.loc[grouped_df.content_id == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'title': titles, 'score': scores})

    return recommendations